# How to add memory of named entities

One strategy for managing long conversation histories is to restrict what information is stored "long-term". For example, instead of storing messages verbatim, an application can persist selected facts that it learns during the conversation.

In [1]:
from langchain_core.documents import Document
from langchain_core.runnables.config import ensure_config
from langchain_core.tools import tool
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.prebuilt import create_react_agent
from typing_extensions import TypedDict


class KnowledgeTriple(TypedDict):
    subject: str
    predicate: str
    object_: str


vector_store = InMemoryVectorStore(OpenAIEmbeddings())

@tool
def update_knowledge_store(knowledge_triple: KnowledgeTriple) -> None:
    """Add a new knowledge triple to the knowledge store."""
    serialized = " ".join(knowledge_triple.values())
    config_dict = ensure_config()
    user_id = config_dict["configurable"].get("user_id")
    document = Document(
        serialized,
        metadata={
            "subject": knowledge_triple["subject"],
            "user_id": user_id,
        },
    )
    vector_store.add_documents([document])


@tool
def fetch_knowledge(query: str) -> str:
    """Fetch facts from the knowledge store."""
    config_dict = ensure_config()
    user_id = config_dict["configurable"].get("user_id")
    def _filter_function(doc: Document) -> bool:
        return doc.metadata.get("user_id") == user_id

    documents = vector_store.similarity_search(
        query, k=3, filter=_filter_function
    )
    return "\n\n".join(document.page_content for document in documents)


tools = [update_knowledge_store, fetch_knowledge]

llm = ChatOpenAI(model="gpt-4o-mini")

system_message = """
You are a friendly assistant. When you are told a fact about a person,
use the `update_knowledge_store` tool to store that fact via a
subject, predicate, object triple.

Only update the knowledge store if you are told a material fact.

If you require facts about a person to answer a query, use the
`fetch_knowledge` tool to retrieve relevant facts.
"""

app = create_react_agent(llm, tools, state_modifier=system_message)

In [2]:
config = {"configurable": {"thread_id": "abc123", "user_id": "user_1"}}
query = "Hi I'm Alice, how are you?"

input_messages = [{"role": "user", "content": query}]
for event in app.stream({"messages": input_messages}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi I'm Alice, how are you?
================================== Ai Message ==================================

Hello Alice! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [3]:
query = "What's my name?"

input_messages = [{"role": "user", "content": query}]
for event in app.stream({"messages": input_messages}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I don't have your name yet. If you tell me, I can remember it for you!


In [4]:
query = "Bob likes apples."

input_messages = [{"role": "user", "content": query}]
for event in app.stream({"messages": input_messages}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Bob likes apples.
================================== Ai Message ==================================
Tool Calls:
  update_knowledge_store (call_nYa8eaRfzsfzQ5p1iBnniprv)
 Call ID: call_nYa8eaRfzsfzQ5p1iBnniprv
  Args:
    knowledge_triple: {'subject': 'Bob', 'predicate': 'likes', 'object_': 'apples'}
================================= Tool Message =================================
Name: update_knowledge_store

null
================================== Ai Message ==================================

I've stored the fact that Bob likes apples. If you have more information or questions, feel free to share!


In [5]:
query = "What does Bob like to eat?"

input_messages = [{"role": "user", "content": query}]
for event in app.stream({"messages": input_messages}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What does Bob like to eat?
================================== Ai Message ==================================
Tool Calls:
  fetch_knowledge (call_mxnUxjOw2PVRfi5lVNgWMC8e)
 Call ID: call_mxnUxjOw2PVRfi5lVNgWMC8e
  Args:
    query: Bob
================================= Tool Message =================================
Name: fetch_knowledge

Bob likes apples
================================== Ai Message ==================================

Bob likes apples.


In [6]:
vector_store.similarity_search("Bob")

[Document(id='bd7486ee-cb8e-4167-ae2b-ef973aa0bb90', metadata={'subject': 'Bob', 'user_id': 'user_1'}, page_content='Bob likes apples')]

In [7]:
from langchain_core.messages import BaseMessage, trim_messages
from langgraph.checkpoint.memory import MemorySaver


def state_modifier(state) -> list[BaseMessage]:
    """Given the agent state, return a list of messages for the chat model."""
    return trim_messages(
        state["messages"],
        token_counter=len,  # specifying `len` will count messages
        max_tokens=4,  # retain up to 5 messages.
        strategy="last",
        start_on=("human", "ai"),
        include_system=True,
        allow_partial=False,
    )


checkpointer = MemorySaver()

app = create_react_agent(
    llm,
    tools,
    state_modifier=state_modifier,
    checkpointer=checkpointer,
)

In [8]:
config = {"configurable": {"thread_id": "abc234", "user_id": "user_1"}}

prior_messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Bob likes apples."},
    {"role": "assistant", "content": "Nice, good to know."},
    {"role": "user", "content": "What is 2+2?"},
    {"role": "assistant", "content": "Four."},
    {"role": "user", "content": "Thanks."},
    {"role": "assistant", "content": "You're welcome!"},
]

query = "Bob is my friend."

input_messages = prior_messages + [{"role": "user", "content": query}]
for event in app.stream({"messages": input_messages}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Bob is my friend.
================================== Ai Message ==================================
Tool Calls:
  update_knowledge_store (call_75RzxpGDvU53vVovS4aImKUv)
 Call ID: call_75RzxpGDvU53vVovS4aImKUv
  Args:
    knowledge_triple: {'subject': 'Bob', 'predicate': 'is a friend of', 'object_': 'User'}
================================= Tool Message =================================
Name: update_knowledge_store

null
================================== Ai Message ==================================

I've stored the fact that Bob is your friend. If you have more information about Bob or anyone else, feel free to share!


In [12]:
query = "What does my friend like to eat? Use the tool."

input_messages = prior_messages + [{"role": "user", "content": query}]
for event in app.stream({"messages": input_messages}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What does my friend like to eat? Use the tool.
================================== Ai Message ==================================
Tool Calls:
  fetch_knowledge (call_tN1mBQgAzJ16lXZ3jQPhHQGk)
 Call ID: call_tN1mBQgAzJ16lXZ3jQPhHQGk
  Args:
    query: friend likes to eat
================================= Tool Message =================================
Name: fetch_knowledge

Bob likes apples

Bob is a friend of User
================================== Ai Message ==================================

Your friend Bob likes to eat apples.
